In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import time
import numpy as np
import torch
import pyzed.sl as sl
import rerun as rr
import time
import kornia as K
import mediapy as media
import matplotlib.pyplot as plt

from ember.corners import compute_corner_map

c:\Users\DOMCE\anaconda3\envs\ember\Lib\site-packages\kornia\feature\lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


In [2]:
# rr.init("checkerboard.ipynb", spawn=True)
# rr.connect()  # Connect to a remote viewer

def sl_image_to_np(sl_image):
    # BRGA to RGB
    image = sl_image.numpy()
    image = image[..., [2, 1, 0, 3]]
    return image

In [3]:
RECORDING_PATH = "C:/Users/DOMCE/OneDrive/Desktop/Personal Projects/embodied-perception/data/zed_recordings/HD720_SN33087127_15-44-16.svo2"

init_parameters = sl.InitParameters()
init_parameters.set_from_svo_file(RECORDING_PATH)

# Open the camera
zed_camera = sl.Camera()
camera_status = zed_camera.open(init_parameters)
if camera_status != sl.ERROR_CODE.SUCCESS: #Ensure the camera opened succesfully 
    print("Camera Open", camera_status, "Exit program.")
    exit(1)

runtime_param = sl.RuntimeParameters()


sl_image_left = sl.Mat()
sl_image_right = sl.Mat()
sl_depth = sl.Mat()

assert zed_camera.grab(runtime_param) == sl.ERROR_CODE.SUCCESS
# Retrieve frame data
frame_idx = zed_camera.get_svo_position()
# zed_camera.

zed_camera.retrieve_image(sl_image_left, sl.VIEW.LEFT)
zed_camera.retrieve_image(sl_image_right, sl.VIEW.RIGHT)
zed_camera.retrieve_measure(sl_depth, sl.MEASURE.DEPTH)  # Get the depth map

image_left=sl_image_to_np(sl_image_left)
image_right=sl_image_to_np(sl_image_right)
depth=sl_depth.numpy()

In [4]:
image_left

array([[[ 70,  67,  60, 255],
        [ 74,  71,  64, 255],
        [ 75,  71,  65, 255],
        ...,
        [ 94,  85,  91, 255],
        [ 94,  85,  87, 255],
        [ 92,  83,  86, 255]],

       [[ 73,  70,  63, 255],
        [ 75,  72,  65, 255],
        [ 77,  74,  67, 255],
        ...,
        [ 94,  85,  90, 255],
        [ 93,  84,  89, 255],
        [ 95,  86,  88, 255]],

       [[ 78,  75,  68, 255],
        [ 78,  75,  68, 255],
        [ 74,  71,  64, 255],
        ...,
        [ 93,  84,  90, 255],
        [ 96,  87,  88, 255],
        [ 93,  84,  87, 255]],

       ...,

       [[ 65,  57,  46, 255],
        [ 65,  57,  46, 255],
        [ 65,  57,  46, 255],
        ...,
        [106, 106, 107, 255],
        [122, 122, 121, 255],
        [161, 161, 161, 255]],

       [[ 65,  57,  46, 255],
        [ 65,  57,  46, 255],
        [ 65,  57,  46, 255],
        ...,
        [106, 106, 107, 255],
        [108, 108, 108, 255],
        [161, 161, 161, 255]],

       [[ 65